In [1]:
import os
import shutil
import glob
import logging
import pathlib
from pathlib import Path
from tempfile import gettempdir, NamedTemporaryFile
from typing import List, Dict, Union

import subprocess

import pandas as pd
import numpy as np

import datamol as dm
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import PandasTools

from openmm.app import PDBFile
from pdbfixer import PDBFixer
from pymol import cmd

from openeye import oechem
from openeye import oequacpac

In [2]:
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    handlers=[
                        logging.StreamHandler()
                    ])
logger = logging.getLogger(__name__)

In [3]:
from Pymol_Docking import Plants_Docking
from Pymol_Docking import Pymol_Docking

In [ ]:
pymol = Pymol_Docking("./LAC3.pdb", "c1ccccc1")
pymol.run_smina_docking("Dock", "XXX")

In [4]:
tests_dir = Path("./tests")
data_dir = tests_dir / "sample_data"

In [5]:
protein_pdb = data_dir / "LAC3.pdb"
protein_pdb = protein_pdb.absolute()

ligands_sdf = data_dir / "Ligand.sdf"
ligands_sdf = ligands_sdf.absolute()

In [6]:
os.chdir(tests_dir)

In [7]:
pymol = Pymol_Docking(protein_pdb, ligands_sdf)

In [ ]:
pymol.run_smina_docking("Dock", "XXX")

In [7]:
plants = Plants_Docking(protein_pdb, ligands_sdf)

In [ ]:
plants.prepare_protein()

In [ ]:
plants.prepare_ligands()

In [ ]:
def openeye_fixer(oemol, explicit_H=True):

    to_edit = oechem.OEGraphMol(oemol)
    
    oechem.OEDetermineConnectivity(to_edit)
    oechem.OEFindRingAtomsAndBonds(to_edit)
    oechem.OEPerceiveBondOrders(to_edit)
    oechem.OEAssignImplicitHydrogens(to_edit)
    oechem.OEAssignFormalCharges(to_edit)
    
    oechem.OEClearAromaticFlags(to_edit)
    
    oechem.OEFindRingAtomsAndBonds(to_edit)
    oechem.OEAssignAromaticFlags(to_edit)
    for bond in to_edit.GetBonds():
        if bond.IsAromatic():
            bond.SetIntType(5)
        elif bond.GetOrder() != 0:
            bond.SetIntType(bond.GetOrder())
        else:
            bond.SetIntType(1)
    
    oechem.OEKekulize(to_edit)
    
    if explicit_H:
        oechem.OEAddExplicitHydrogens(to_edit)
        oequacpac.OEGetReasonableProtomer(to_edit)
    
    return oechem.OEGraphMol(to_edit)

In [ ]:
class Pymol_Docking:
    def __init__(self, protein_pdb: str, ligands_sdf: str):
        self.workdir: Path = Path(os.getcwd())
        self.protein_pdb: Path = Path(protein_pdb)
        self.crystal_sdf: Path = Path("Crystal.sdf")
        
        self.ligands_sdf: Path = Path(ligands_sdf)
    
    def prepare_protein(self) -> Path:
        protein_basename: str = self.protein_pdb.stem
        protein_PREP: Path = self.workdir / f"{protein_basename}_PREP.pdb"

        fixer = PDBFixer(filename=str(self.protein_pdb))
        fixer.removeHeterogens(keepWater=False)
        fixer.findMissingResidues()
        fixer.findMissingAtoms()
        fixer.findNonstandardResidues()

        fixer.addMissingAtoms()
        fixer.addMissingHydrogens(7.4)

        PDBFile.writeFile(fixer.topology, fixer.positions, open(protein_PREP.as_posix(), 'w'))
        return protein_PREP
    
    @staticmethod
    def fix_3d_mol(sdf_file: Path, TMP_basename: str) -> Path:
        TMP_fixed_sdf = Path(gettempdir()) / f"{TMP_basename}_fixed.sdf"
        
        ifs = oechem.oemolistream()
        ofs = oechem.oemolostream(TMP_fixed_sdf.as_posix())
        
        ifs.open(sdf_file.as_posix())
        for oemol in ifs.GetOEGraphMols():
            fixed_oemol = openeye_fixer(oemol)
            oechem.OEWriteMolecule(ofs, fixed_oemol)
        
        return TMP_fixed_sdf
    
    def prepare_ligands(self):
        fixed_crystal: Path = self.fix_3d_mol(self.crystal_sdf, "crystal")
        fixed_ligands: Path = self.fix_3d_mol(self.ligands_sdf, "ligand")
        
        return fixed_ligands, fixed_crystal

    def run_smina_docking(self, mode: str, docked_basename: str) -> Path:
        # Fix the protein
        print("Running protein preparation")
        protein_PKA: Path = self.prepare_protein()
        
        # Fix the ligands
        fixed_ligands, fixed_crystal = self.prepare_ligands()
        
        # Define the output
        smina_output: Path = self.workdir / f"{docked_basename}.sdf"
        smina_log = smina_output.with_suffix(".log")
        
        smina_lst = [
            "smina",
            "-r", protein_PKA.as_posix(),
            "-l", fixed_ligands.as_posix(),
            "--autobox_ligand", fixed_crystal.as_posix(),
            "-o", smina_output,
            "--exhaustiveness", "16"
        ]
        
        print("Running docking")
        with open(smina_log, "w") as log_file:
            subprocess.run(smina_lst, check=True, stdout=log_file, stderr=log_file)
            os.remove(smina_log)
        
        return smina_output

In [ ]:
pymol_docking = Pymol_Docking("LAC3.pdb", "F5_prep.sdf")
pymol_docking.prepare_protein()

In [ ]:
@cmd.extend
def on_site_docking(protein_selection, ligand_selection, outname: str):
    protein_name = cmd.get_object_list(protein_selection)[0]
    ligand_name = cmd.get_object_list(ligand_selection)[0]
    
    to_save_protein = Path(gettempdir()) / f"{protein_name}.pdb"
    to_save_ligand = Path(gettempdir()) / f"{ligand_name}.sdf"
    
    cmd.save(str(to_save_protein), protein_selection)
    cmd.save(str(to_save_ligand), ligand_selection)
    
    pymol_docking = Pymol_Docking(str(to_save_protein), str(to_save_ligand))
    docked_sdf: Path = pymol_docking.run_smina_docking("smina", outname)
    
    cmd.load(str(docked_sdf), f"{ligand_name}_Docked")